In [ ]:
!git clone https://github.com/matzim95/ML-datasets

fatal: destination path 'ML-datasets' already exists and is not an empty directory.


In [ ]:
import pandas as pd

def load_dataset(filename, class_column, index_col=None):
    dataset = pd.read_csv(f'ML-datasets/{filename}.csv', index_col=index_col)
    dataset['class'] = dataset[class_column].astype('category').cat.codes
    classes = dataset.pop(class_column).unique()
    return dataset, classes

def prepare_dataset(dataset_name):
    params = {'iris': {'class_column': 'species', 'index_col': None},
              'wine': {'class_column': 'Class', 'index_col': None},
              'glass': {'class_column': 'Type', 'index_col': 'ID'}}
    dataset, classes = load_dataset(dataset_name,
                                    **params[dataset_name])
    y = dataset.pop('class')
    X = dataset
    df = pd.concat([X, y], axis=1)
    return X, y, classes, df

In [ ]:
X, y, classes, df = prepare_dataset('glass')
df

,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,class
ID,,,,,,,,,,
1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,0
2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,0
3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,0
4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,0
5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,0
...,...,...,...,...,...,...,...,...,...,...
210,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,3
211,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,3
212,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,3


In [ ]:
from sklearn.preprocessing import  StandardScaler, MinMaxScaler

def preprocess_data(X, preprocessor=None):
    if preprocessor is not None:
        preprocessor.fit(X)
        X_preprocessed = X.copy()
        X_preprocessed[:] = preprocessor.transform(X.values)
        X = X_preprocessed
    return X

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# average:
#   None - wyniki dla każdej klasy osobno
#   'binary' - dla wybranej w `pos_label` etykiety
#   'micro' - dla całkowitej liczby TP, FN, FP
#   'macro' - nieważona średnia dla każdej klasy
#   'weighted' - ważona średnia dla każdej klasy

def calculate_metrics(target, prediction, average='macro'):
    accuracy = accuracy_score(target, prediction)
    precision = precision_score(target, prediction, average=average)
    recall = recall_score(target, prediction, average=average)
    f1 = f1_score(target, prediction, average=average)
    mislabeled = (target != prediction).sum()
    total = len(target)
    return accuracy, precision, recall, f1, mislabeled, total

def print_results(metrics, classifier_id='classifier'):
    print(f'Results for {classifier_id}')
    print('----')
    print(f'  Accuracy:  {metrics[0]}')
    print(f'  Precision: {metrics[1]}')
    print(f'  Recall:    {metrics[2]}')
    print(f'  F1 score:  {metrics[3]}')
    print(f'  Mislabeled {metrics[4]} out of {metrics[5]}')
    print('\n')
    


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Normalizer, StandardScaler


In [ ]:
normalizer = Normalizer()
standarizer = StandardScaler()
preprocessor = standarizer

In [ ]:
X = preprocess_data(X, standarizer)
X.describe()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron
count,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02
mean,-2.877449e-14,2.179980e-15,-2.801497e-16,-3.434428e-16,9.966067e-16,7.470660e-17,-3.137418e-16,-1.763906e-16,-6.121791e-17
std,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00
min,-2.381516e+00,-3.286943e+00,-1.865511e+00,-2.318616e+00,-3.676472e+00,-7.639186e-01,-2.484084e+00,-3.528768e-01,-5.864509e-01
25%,-6.082728e-01,-6.141580e-01,-3.957744e-01,-5.117560e-01,-4.800288e-01,-5.756501e-01,-5.049657e-01,-3.528768e-01,-5.864509e-01
50%,-2.262293e-01,-1.323817e-01,5.527787e-01,-1.704602e-01,1.799655e-01,8.905322e-02,-2.514132e-01,-3.528768e-01,-5.864509e-01
75%,2.614331e-01,5.120326e-01,6.361680e-01,3.715977e-01,5.649621e-01,1.735820e-01,1.518057e-01,-3.528768e-01,4.422417e-01
max,5.137232e+00,4.875637e+00,1.254639e+00,4.125851e+00,3.570524e+00,8.780145e+00,5.094318e+00,5.997210e+00,4.659881e+00


In [ ]:

import xgboost
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# le = LabelEncoder() # zamienia tekst na liczby
# df = df.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')


In [ ]:
y.value_counts()

1    76
0    70
3    29
5    17
2    13
4     9
Name: class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
 X, y, stratify=y, test_size=0.33, random_state=42)

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([

                   ('classifier', RandomForestClassifier())])


param = {'classifier__n_estimators': [1, 10, 20,50],
             }

grid_11 = GridSearchCV(pipe, param,cv=4, return_train_score=False,verbose=True, n_jobs=-1)
grid_11.fit(X_train, y_train)
print(grid_11.best_params_)
print("Best parameter (CV score=%0.3f):" % grid_11.best_score_)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
{'classifier__n_estimators': 50}
Best parameter (CV score=0.748):


In [ ]:
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_pred, y_test, average='macro')
calculate_metrics(y_test, y_pred)

(0.7887323943661971,
 0.7586111111111111,
 0.7643478260869565,
 0.7463813303099017,
 15,
 71)

In [ ]:
y.value_counts()

1    76
0    70
3    29
5    17
2    13
4     9
Name: class, dtype: int64

In [ ]:
df['class']

ID
1      0
2      0
3      0
4      0
5      0
      ..
210    3
211    3
212    3
213    3
214    3
Name: class, Length: 214, dtype: int8

In [ ]:
df

,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,class
ID,,,,,,,,,,
1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,0
2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,0
3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,0
4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,0
5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,0
...,...,...,...,...,...,...,...,...,...,...
210,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,3
211,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,3
212,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,3


In [ ]:
from sklearn.utils import resample
df_majority0 = df[df['class'] == 1]
df_majority1 = df[df['class'] == 0]
df_majority2 = df[df['class'] == 3]
df_minority0  = df[df['class'] == 5]
df_minority1  = df[df['class'] == 2]
df_minority2  = df[df['class'] == 4]



In [ ]:

# Upsample minority class
df_majority0_upsampled = resample(df_majority0, 
                                 replace=False,    # sample with replacement
                                 n_samples=76,    # to match majority class
                                 random_state=42)
df_majority1_upsampled = resample(df_majority1,
                                 replace=False,    # sample with replacement
                                 n_samples=70,    # to match majority class
                                 random_state=42) # reproducible results

df_majority2_upsampled = resample(df_majority2, 
                                 replace=True,    # sample with replacement
                                 n_samples=40,    # to match majority class
                                 random_state=42) # reproducible results
df_minority0_downsampled = resample(df_minority0, 
                                 replace=True,    # sample with replacement
                                 n_samples=30,    # to match majority class
                                 random_state=42) # reproducible results
df_minority1_downsampled = resample(df_minority1, 
                                 replace=True,    # sample with replacement
                                 n_samples=20,    # to match majority class
                                 random_state=42) # reproducible results
df_minority2_downsampled = resample(df_minority2, 
                                 replace=True,    # sample with replacement
                                 n_samples=17,    # to match majority class
                                 random_state=42) # reproducible results
 
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority0_upsampled, df_majority1_upsampled, df_majority2_upsampled, df_minority0_downsampled, df_minority1_downsampled, df_minority2_downsampled])
 
# Display new class counts
df_upsampled['class'].value_counts()

1    76
0    70
3    40
5    30
2    20
4    17
Name: class, dtype: int64

In [ ]:
X = df_upsampled.iloc[:, :-1]
y = df_upsampled['class']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
 X, y, test_size=0.33, random_state=42)

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([

                   ('classifier', RandomForestClassifier())])


param = {'classifier__n_estimators': [1, 10, 20,50],
             }

grid_11 = GridSearchCV(pipe, param,cv=5, return_train_score=False,verbose=True, n_jobs=-1)
grid_11.fit(X_train, y_train)
grid_11.best_params_
print("Best parameter (CV score=%0.3f):" % grid_11.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


Best parameter (CV score=0.888):


In [ ]:
y_pred = grid_11.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, y_pred,average='macro')

0.7906540319583798

In [ ]:
from sklearn import  metrics


models = []

models.append(('RandomForest', grid_11.best_estimator_))


precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test),average='macro') ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test),average='macro') ))
    print("f1_score: {}".format( metrics.f1_score(y_train, model.predict(X_train),average='macro') ))
    print("f1_score test: {}".format( metrics.f1_score(y_test, model.predict(X_test),average='macro') ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_train, model.predict(X_train)) ))
    print("accuracy_score_test: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))

RandomForest
precision_score: 0.7906540319583798
recall_score: 0.7688405797101449
f1_score: 0.9904453690861459
f1_score test: 0.7650793650793651
accuracy_score: 0.993006993006993
accuracy_score_test: 0.8169014084507042


Z.8. Na zbiorze glasses wykonać zrównoważnie klas  a nastepnie użyć metody Random Forest albo Xgboost zapisze wynik f1_score na zbiorze testowym.*kursywa*